In [27]:
from pearce.emulator.emu import OriginalRecipe, ExtraCrispy
from pearce.emulator.trainingData import parameter, PARAMS

In [28]:
from george.kernels import ExpSquaredKernel
import george

In [29]:
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()
from itertools import cycle
colors = cycle(sns.color_palette())

In [30]:
training_dir = '/u/ki/swmclau2/des/PearceLHC/'

In [31]:
or_params = PARAMS[:]
or_params.append(parameter('r', 0, 1)) #95% sure bounds aren't used for r
emu = OriginalRecipe(training_dir,or_params)
#emu = ExtraCrispy(training_dir)

In [32]:
ig = emu.get_initial_guess('xi', {})
a = ig['amp']
kernel = a * ExpSquaredKernel(emu.metric, ndim=emu.emulator_ndim)
loo_gp = george.GP(kernel)
# gp = george.GP(kernel, solver=george.HODLRSolver, nleaf=x.shape[0]+1,tol=1e-18)

loo_gp.compute(emu.x[1:], emu.yerr[1:])  # NOTE I'm using a modified version of george!

In [33]:
K_inv_full = emu.gp.solver.apply_inverse(np.eye(emu.x.shape[0]),in_place=True)

In [34]:
y_param = 'logMmin'
emulation_point = [('f_c', 0.233), ('logM0', 12.0), ('sigma_logM', 0.533), 
                   ('alpha', 1.083),('logM1', 13.5), ('logMmin', 12.233)]
em_params = dict(emulation_point)
#del em_params[y_param]

In [35]:
from os import path
save_dir = '/u/ki/swmclau2/des/EmulatorMCMC/'
rpoints = np.log10(np.loadtxt(path.join(save_dir, 'rpoints.npy')) )
true_vals = np.log10( np.loadtxt(path.join(save_dir, 'xi.npy')) )

In [36]:
rbins = np.array([  0.06309573,   0.12437607,   0.24517359,   0.34422476, 0.48329302, 0.67854546,\
           0.9526807 , 1.33756775,1.8779508 ,   2.6366509 ,   3.70186906,   5.19743987, 7.29722764, \
           10.24533859,  14.38449888,  20.1958975 , 28.35512583,  39.81071706] )
#rpoints= np.log10((rbins[1:]+rbins[:-1])/2)
em_params['r'] = rpoints#[:2]

In [37]:
input_params = {}
input_params.update(em_params)

# i'd like to remove 'r'. possibly requiring a passed in param?
t_list = [input_params[p.name] for p in emu.ordered_params]
t_grid = np.meshgrid(*t_list)
t = np.stack(t_grid).T
t = t.reshape((-1, emu.emulator_ndim))
try:
    t = np.sort(t.view(','.join(['float64' for _ in xrange(min(t.shape))])),
                order=['f%d' % i for i in xrange(min(t.shape))], axis=0).view(np.float)
except ValueError:
        t = np.sort(t.view(','.join(['float64' for _ in xrange(max(t.shape))])),
                order=['f%d' % i for i in xrange(max(t.shape))], axis=0).view(np.float)

/afs/slac.stanford.edu/u/ki/swmclau2/.local/lib/python2.7/site-packages/ipykernel/__main__.py:10: DeprecationWarning: Changing the shape of non-C contiguous array by
descriptor assignment is deprecated. To maintain
the Fortran contiguity of a multidimensional Fortran
array, use 'a.T.view(...).T' instead
/afs/slac.stanford.edu/u/ki/swmclau2/.local/lib/python2.7/site-packages/ipykernel/__main__.py:13: DeprecationWarning: Changing the shape of non-C contiguous array by
descriptor assignment is deprecated. To maintain
the Fortran contiguity of a multidimensional Fortran
array, use 'a.T.view(...).T' instead
/afs/slac.stanford.edu/u/ki/swmclau2/.local/lib/python2.7/site-packages/ipykernel/__main__.py:14: DeprecationWarning: Changing the shape of non-C contiguous array by
descriptor assignment is deprecated. To maintain
the Fortran contiguity of a multidimensional Fortran
array, use 'a.T.view(...).T' instead


In [38]:
print t.shape

(17, 7)


In [39]:
print emu.y.shape

(8432,)


In [40]:
#loo_mu = loo_gp.predict(emu.y[1:, 0], t , mean_only=True)
loo_mu = loo_gp.predict(emu.y[1:], t , mean_only=True)
print loo_mu

[ 3.3343133   2.92986007  2.51030092  2.2116945   1.88773449  1.49316321
  1.09248502  0.74583087  0.48338015  0.25918154  0.01873291 -0.20661966
 -0.47061312 -0.74351721 -1.03990018 -1.4047761  -1.94146127]


In [41]:
loo_inv = loo_gp.solver.apply_inverse(np.eye(emu.x.shape[0]-1),in_place=True)
print loo_inv.shape, K_inv_full.shape

(8431, 8431) (8432, 8432)


In [42]:
print loo_inv[0,:]
print 
print loo_inv[1,:]

[  1.96564257e+02  -2.74440036e+02  -4.17402233e+01 ...,   4.86810266e-03
   4.65590985e-03  -4.36067291e-03]

[ -2.74440036e+02   7.61615949e+02   6.14895863e+01 ...,  -1.88435473e-02
  -2.50623516e-02   2.21037310e-02]


In [43]:
print K_inv_full[0,:]
print 
print K_inv_full[1,:]

[  1.07207315e+02  -7.49339987e+01   1.00697096e+02 ...,   1.87325557e-02
  -4.71164214e-03  -3.16211120e-04]

[ -7.49339987e+01   2.81840050e+02  -4.42344406e+02 ...,  -9.56717340e-02
   2.47047181e-02   1.04994773e-03]


In [44]:
mu, G = emu.emulate(em_params)
print mu

[ 3.33430587  2.92689423  2.49717286  2.20846211  1.87173901  1.48554091
  1.09514167  0.75355011  0.48323972  0.25937854  0.02380753 -0.20503501
 -0.47126951 -0.73669219 -1.04039574 -1.40426527 -1.90917015]


In [45]:
yhat = loo_gp.predict(emu.y[1:], emu.x[0].reshape((1,emu.x.shape[1])), mean_only=True)
#yhat = loo_gp.predict(emu.y[1:, 0], emu.x[0].reshape((1,emu.x.shape[1])), mean_only=True)

print yhat

[ 3.81843655]


In [46]:
#print emu.y[0, 0]
print emu.y[0]

3.72177668084


In [47]:
x, y = emu.x[:], emu.y[:]

N = K_inv_full.shape[0]

In [48]:
from time import time
from scipy.linalg import inv

In [49]:
my_inds = [i for i in xrange(1, N-1)]
my_inds.append(0)
my_inds = np.array(my_inds)

In [50]:
from copy import deepcopy

In [51]:
times = np.zeros((N,))

In [52]:
mus = np.zeros((N, len(rpoints)))

In [55]:
for idx in xrange(N):
    print idx
    t0 = time()
    #TODO put switches at the end of the loop too
    #then i can make just one deepcopy
    #t_x = deepcopy(emu.x)
    t_x = emu.x
    t_x[[N-1, idx]] = t_x[[idx, N-1]]
    
    #t_yerr = deepcopy(emu.yerr)
    t_yerr = emu.yerr
    t_yerr[[N-1, idx]] = t_yerr[[idx, N-1]]
    
    #t_y = deepcopy(emu.y[:, 0])
    t_y = emu.y
    #t_y[[N-1, idx], :] = t_y[[idx, N-1], :]
    t_y[[N-1, idx]] = t_y[[idx, N-1]]
    #K_inv = deepcopy(K_inv_full)
    K_inv = K_inv_full
    
    K_inv[[idx, N - 1], :] = K_inv[[N - 1, idx], :]
    K_inv[:, [idx, N - 1]] = K_inv[:, [N - 1, idx]]
    
    K_m_idx_inv = K_inv[:N - 1, :][:,:N - 1] \
            -np.outer(K_inv[N-1,:N-1], K_inv[:N-1,N-1])/K_inv[N-1,N-1]
           
    #alpha_m_idx = [np.dot(K_m_idx_inv, t_y[:N-1, rbin ]-emu.gp.mean(t_x[:N-1])) for rbin in xrange(t_y.shape[1]) ]
    alpha_m_idx = np.dot(K_m_idx_inv, t_y[:N-1 ]-emu.gp.mean(t_x[:N-1]))

    Kxxs_t = emu.gp.kernel.value(t, t_x[:N-1])
    #mus[idx, :] = np.array([np.dot(Kxxs_t, alpha_m_idx[rbin])+ emu.gp.mean(t) for rbin in xrange(t_y.shape[1]) ])[:,0]
    mus[idx, :] = np.dot(Kxxs_t, alpha_m_idx)+ emu.gp.mean(t)

    #print mus[idx]
    print time()-t0, 's'
    #print
    K_inv_full[[idx, N - 1], :] = K_inv_full[[N - 1, idx], :]
    K_inv_full[:, [idx, N - 1]] = K_inv_full[:, [N - 1, idx]]

    t_x[[N-1, idx]] = t_x[[idx, N-1]]
    t_yerr[[N-1, idx]] = t_yerr[[idx, N-1]]
    #t_y[[N-1, idx],:] = t_y[[idx, N-1],:]
    t_y[[N-1, idx]] = t_y[[idx, N-1]]
    
    break

0
8.82348203659 s
1
9.3327088356 s
2
9.63330483437 s
3
10.690759182 s
4
12.0796670914 s
5
9.99576306343 s
6
9.35908699036 s
7
9.60755586624 s
8
9.14417695999 s
9
9.08317995071 s
10
9.40858602524 s
11
9.24272012711 s
12
8.76469802856 s
13
9.21109509468 s
14
9.77507805824 s
15
9.0501768589 s
16
9.50348591805 s
17
9.28961086273 s
18
9.01425099373 s
19
9.27239203453 s
20
9.35653495789 s
21
8.94424104691 s
22
9.21178007126 s
23
9.36036586761 s
24
8.95948600769 s
25
9.26210904121 s
26
8.79289293289 s
27
9.3957118988 s
28
9.23135495186 s
29
8.95174503326 s
30
9.49422502518 s
31
9.33979201317 s
32
8.98230099678 s
33
9.30542206764 s
34
9.28829717636 s
35
8.99854183197 s
36
9.35173511505 s
37
9.41651678085 s
38
8.94318199158 s
39


KeyboardInterrupt: 

In [58]:
for idx in xrange(N):
    print idx
    t0 = time()
    #TODO put switches at the end of the loop too
    #then i can make just one deepcopy
    #t_x = deepcopy(emu.x)
    t_x = emu.x
    t_x[[N-1, idx]] = t_x[[idx, N-1]]
    
    #t_yerr = deepcopy(emu.yerr)
    t_yerr = emu.yerr
    t_yerr[[N-1, idx]] = t_yerr[[idx, N-1]]
    
    #t_y = deepcopy(emu.y[:, 0])
    t_y = emu.y
    #t_y[[N-1, idx], :] = t_y[[idx, N-1], :]
    t_y[[N-1, idx]] = t_y[[idx, N-1]]
    loo_gp.compute(t_x[:N-1], t_yerr[:N-1])  
    mus[idx,:] = loo_gp.predict(emu.y[:N-1], t , mean_only=True)

    print time()-t0, 's'
    #print
    K_inv_full[[idx, N - 1], :] = K_inv_full[[N - 1, idx], :]
    K_inv_full[:, [idx, N - 1]] = K_inv_full[:, [N - 1, idx]]

    t_x[[N-1, idx]] = t_x[[idx, N-1]]
    t_yerr[[N-1, idx]] = t_yerr[[idx, N-1]]
    #t_y[[N-1, idx],:] = t_y[[idx, N-1],:]
    t_y[[N-1, idx]] = t_y[[idx, N-1]]

0
0.000204086303711
17.5301651955 s
1
3.19480895996e-05
16.7995769978 s
2
3.09944152832e-05


KeyboardInterrupt: 

In [ ]:
#print mus[:, 0]

In [ ]:
(N-1)/float(N)

In [ ]:
(N-1.0)/N*np.cov(mus, rowvar=False)

In [ ]:
print G